## Applied Statistics final SGA, by Magomedov Rustam

Task description:
**We would like to estimate the effect of the treatment on Growth and dANB by taking into account causal relationships between variables. Our knowledge of those causal relationships is represented on the following DAG**

Given the graph, select the variables to condition on, apply suitable adjustment method, and calculate your estimates of causal effects: Treatment on Growth, Treatment on dANB, both ATE and ATET.

Your analysis should contain:
- Selection of covariates to adjust for (informed by the graph)
- Application of the most suitable adjustment method
- Estimates of the ATE and ATET

The maximal grade for the assignment is 25 points , where for each of two estimated causal effects you could get:
- a maximum of 6.5 points for correct and optimal selection of the adjustment set,
- a maximum of 3 points for correct estimation of the ATE,
- a maximum of 3 points for correct estimation of the ATET.

First of all, let us discss the elements of the directed graph. 

For case when 
- Y = `Growth` 
- T = `Treatment`

1. Firstly, one can see that the `treatment` element has no direct path to `growth`. Based on the front-door adjustment formula, there exists a backdoor path from `treatment` to `growth`. Howerver, such undirected path through the unobserved confounders is present, but the data on these UC is absent from the `malocclusion.csv` data. Hence, we can conclude that calculating ATE and ATET can be proclaimed either statistically insignificant or simply say its 0.
2. Secondly, the adjustment set for `treatment` on `growth` is {`dT`}, since `dT` is a descendant of `treatment` and is not a descendant of `growth`.

For case when
- Y = `dANB`
- T = `Treatment`

1. Firstly, `treatment` has a direct effect on `dANB`. Likewise, `growth` element directly influences `dANB`, making `dANB` element a collider. Moreover, `dT` has direct a direct path to `dANB`, which higlights that we have to test for dependence between `dT` & `dANB`, in which case `growth` serves as a mediator. $\therefore$ adjusting for `dT` becomes crucial.
2. Secondly, the valid adjustment set for `Treatment` on `dANB` is {`dT`, `growth`} . Based on the causal effect formula, we woulde have also count `unobserved confounders` as a parent for `Treatment`. However, we cannot adjust to that, since we don't have it in our dataset. Based on the definition of adjustment set, we are left with {`dT`, `growth`} set, since such a set contains neither descendants on the directed path nor is on the non-directed path.



## Libs & udfs

In [1]:
import pandas as pd
from causalinference import CausalModel
from statsmodels.formula.api import ols
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings('ignore')

# global param
treatment = 'Treatment'

def run_exp(df, treatment, outcome, cov):
    """
    runs the ols regression for the given treatment, outcome and covariates params
    """
    # create the formula for the OLS reg
    form = outcome + ' ~ ' + treatment + ' + ' + ' + '.join(cov)
    print(f"Formula: {form}")

    # run the reg
    m = ols(form, df).fit()
    print(m.summary())
    return None

## EDA

In [2]:
mal = pd.read_csv('./malocclusion.csv')
mal.head()

,dANB,dPPPM,dIMPA,dCoA,dGoPg,dCoGo,dT,Growth,Treatment
0,-3.2,-1.1,-4.2,1.0,4.0,3.7,5,0,0
1,-0.6,-0.5,3.8,2.6,-0.1,1.4,3,1,0
2,-1.6,-3.1,-6.0,4.3,4.2,7.1,5,0,0
3,-1.1,-2.1,-12.1,14.1,20.7,17.5,9,0,0
4,-1.1,0.0,-6.7,7.7,8.8,11.0,5,0,0


In [3]:
mal.describe()

,dANB,dPPPM,dIMPA,dCoA,dGoPg,dCoGo,dT,Growth,Treatment
count,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000
mean,-0.227273,-1.374825,-0.785315,5.987413,7.730769,6.732867,4.706294,0.405594,0.461538
std,1.826225,2.715046,5.080894,4.469692,5.532417,4.595141,2.550427,0.492733,0.500271
min,-5.100000,-9.300000,-19.000000,-0.900000,-1.400000,-2.600000,1.000000,0.000000,0.000000
25%,-1.350000,-2.750000,-3.450000,1.800000,3.200000,3.050000,3.000000,0.000000,0.000000
50%,-0.300000,-1.400000,-0.400000,5.500000,6.200000,6.300000,4.000000,0.000000,0.000000
75%,0.950000,0.050000,2.100000,9.750000,12.750000,10.350000,6.000000,1.000000,1.000000
max,4.900000,6.500000,12.000000,20.000000,23.300000,17.500000,12.000000,1.000000,1.000000


## Estimating effect of Treatment on Growth

In [4]:
# params
outcome_gr = 'Growth'
cov_gr = ['dT']

Even though we have no direct path from `treatment` to `growth`, we can still find ATE & ATET. Let us use the OLS regression to do that.

where $d_{T}$ is the set of all possible values of $dT$.

In [5]:
# naive approach
print(f'Naive estimation of Treatment effect on Growth: {mal.Growth[mal.Treatment == 1].mean() - mal.Growth[mal.Treatment == 0].mean()}')  # this estimate is biased because of the confounding variable (growth)

Naive estimation of Treatment effect on Growth: 0.1471861471861472


In [6]:
# run the OLS regression
run_exp(mal, treatment=treatment, outcome=outcome_gr, cov=cov_gr)

Formula: Growth ~ Treatment + dT
                            OLS Regression Results                            
Dep. Variable:                 Growth   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     3.433
Date:                Sun, 22 Oct 2023   Prob (F-statistic):             0.0350
Time:                        20:08:59   Log-Likelihood:                -97.770
No. Observations:                 143   AIC:                             201.5
Df Residuals:                     140   BIC:                             210.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.46

Even though the linear regression model above shows that the effect of `treatment` on `growth` is statistically significant, we still know from our DAG that there is no direct path from `treatment` to `growth`. Hence, we can conclude that the effect of `treatment` on `growth` is 0. Moreover, both $R^2$ and adjusted $R^2$ are very close to 0, which further proves that the effect of `treatment` on `growth` is practically absent.

Now lets check ATE and ATET for `treatment` on `growth`. With OLS we obtained the ATE = 0.208, let us compare it with the results of CausalModel from causalinference lib.

In [7]:
Y = mal[outcome_gr].values  # outcome
D = mal[treatment].values  # treatment
X = mal[cov_gr].values  # covariates

# calling the ols model
causal = CausalModel(Y, D, X)
causal.est_via_ols()
print(causal.estimates)


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.202      0.087      2.338      0.019      0.033      0.372
           ATC      0.190      0.101      1.883      0.060     -0.008      0.387
           ATT      0.217      0.084      2.581      0.010      0.052      0.382



We obtain the same result, which is 0.208. ATET = 0.217. As was discussed above - **such results are analytically correct, but statistically isnignificant.**

## Estimating effect of Treatment on dANB

In [8]:
# params
outcome_danb = 'dANB'
cov_danb = ['Growth', 'dT']

In [9]:
# naive estimate of the effect of the treatment
print(f'Naive estimation of Treatment effect on dANB: {mal.dANB[mal.Treatment == 1].mean() - mal.dANB[mal.Treatment == 0].mean()}')  # this estimate is biased because of the confounding variable (growth)

Naive estimation of Treatment effect on dANB: 2.0287878787878784


Let us start with the `Treatment` on `dANB` causal effect. We will also adjust for `dT` and `Growth` elements, since it is a valid adjustment set for the `Treatment` element.

In [10]:
# modelling
run_exp(df=mal, 
        treatment=treatment, 
        outcome=outcome_danb, 
        cov=cov_danb)

Formula: dANB ~ Treatment + Growth + dT
                            OLS Regression Results                            
Dep. Variable:                   dANB   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.397
Method:                 Least Squares   F-statistic:                     32.12
Date:                Sun, 22 Oct 2023   Prob (F-statistic):           7.78e-16
Time:                        20:08:59   Log-Likelihood:                -250.87
No. Observations:                 143   AIC:                             509.7
Df Residuals:                     139   BIC:                             521.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   

Since `dT` has a p-value of 0.45, we can say that it is not statistically significant. Hence, we can say that the `dT` element is not a confounder for the `Treatment` element, which means that we can estimate the causal effect of `Treatment` on `dANB` without adjusting for `dT`, as was inferred from the DAG.

In [11]:
# remove dT from the covariates, adjust the formula
run_exp(df=mal,
        treatment=treatment, 
        outcome=outcome_danb, 
        cov=cov_danb[:-1])

Formula: dANB ~ Treatment + Growth
                            OLS Regression Results                            
Dep. Variable:                   dANB   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.398
Method:                 Least Squares   F-statistic:                     48.04
Date:                Sun, 22 Oct 2023   Prob (F-statistic):           1.31e-16
Time:                        20:08:59   Log-Likelihood:                -251.17
No. Observations:                 143   AIC:                             508.3
Df Residuals:                     140   BIC:                             517.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.

The OLS regression above suggests that the causal effect of both `Treatment` and `Growth` on `dANB` < 0.05, which is statistically significant. Hence, we can say that both `Treatment` and `Growth` have a causal effect on `dANB`. We also see adjustment for `dT` does not change the ATE or $R^2$ results significantly, as was expected based on the results of the previous regression.

We can also run the propensity score weighting adjustment method to check whether the results are similar to the OLS regression.

In [12]:
# now lets run the propensity score matching
cls = LogisticRegression()
cls = CalibratedClassifierCV(cls)

X = mal[['Growth']]
y = mal['Treatment']
cls.fit(X, y)
mal['e'] = cls.predict_proba(X)[:,1].tolist()
mal.head()

# calculate the weights
mal['w'] = mal['Growth'] / mal['e'] + (1 - mal['Growth']) / (1 - mal['e'])

# run the regression
formula = outcome_danb + ' ~ ' + treatment + ' + ' + ' + '.join(cov_danb[:-1])
m = smf.wls(formula, data=mal, weights=mal['w'])
fitted = m.fit()
print(fitted.summary())

                            WLS Regression Results                            
Dep. Variable:                   dANB   R-squared:                       0.415
Model:                            WLS   Adj. R-squared:                  0.407
Method:                 Least Squares   F-statistic:                     49.68
Date:                Sun, 22 Oct 2023   Prob (F-statistic):           4.97e-17
Time:                        20:08:59   Log-Likelihood:                -251.35
No. Observations:                 143   AIC:                             508.7
Df Residuals:                     140   BIC:                             517.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.5578      0.193     -8.062      0.0

Although ATE is almost identical, we see a slight improvement in $R^2$ and adjusted $R^2$ values. The causal effect of `Treatment` on `dANB` is still statistically significant.

Let us test the matching adjustment method on the data from the previous task to see if it is a better approach to estimating the causal effects.

In [13]:
Y = mal[outcome_danb].values  # outcome
D = mal[treatment].values  # treatment
X = mal[cov_danb[:-1]].values  # covariates

# calling the model using matching adjustment method, adjusting for bias
causal = CausalModel(Y, D, X)
causal.est_via_matching(weights='maha', bias_adj=True)
print(causal.estimates)


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      1.856      0.237      7.829      0.000      1.392      2.321
           ATC      1.860      0.240      7.761      0.000      1.390      2.330
           ATT      1.852      0.240      7.723      0.000      1.382      2.322



As can be inferred from the results above, the matching method suggests that the causal effect of `Treatment` on `dANB` < 0.05 $\implies$ statistically significant.

Moreover, the matching method suggests that the causal effect of `Growth` on `dANB` < 0.05 $\implies$ statistically significant.

Both the ATE and ATET practically do not differ from OLS regression results, which is a good sign. Hence, we can say that both methods prove to be good approaches towards estimating the causal effects in our case.